In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk import word_tokenize
import nltk
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('stopwords')
from nltk.corpus import stopwords as stopwords_scratch
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve,auc  
from sklearn.model_selection import KFold

In [ ]:
df_nusax_test = pd.read_csv(r'../../data/nusax/test.csv', header= None)
df_nusax_train = pd.read_csv(r'../../data/nusax/train.csv', header= None)
df_nusax_valid = pd.read_csv(r'../../data/nusax/valid.csv', header= None)

df_nusax = df_nusax_train.append(df_nusax_test, ignore_index=True)
df_nusax = df_nusax.append(df_nusax_valid, ignore_index=True)

df_nusax.columns = ['id','text','label']
df_nusa = df_nusax.drop(columns=['id'])

df_tambahan = df_nusa.drop(index=0)
df_tambahan.head()

In [ ]:
df_tugas= pd.read_csv(r'../../data/train_preprocess.tsv.txt', sep='\t', header=None)
df_tugas.columns =['text', 'label']
df_tugas.head()

In [ ]:
df = df_tambahan.append(df_tugas, ignore_index=True)
df.drop(df.index[df['label'] == 'label'], inplace = True)
df.head()

In [ ]:
# Chart Klasifikasi Sentimen
df["label"].value_counts().plot.bar(figsize = (7, 4))
plt.title('Grafik Klasifikasi Sentimen')
plt.xlabel('Label')
plt.ylabel('Jumlah Kata')
plt.legend()
plt.show()

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
stop_words = set(stopwords.words('indonesian'))

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

alay_dict = pd.read_csv(r'../../data/new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})

id_stopword_dict = pd.read_csv(r'../../data/stopwordbahasa.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('user',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text
    
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

def remove_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

def stemming(text):
    return stemmer.stem(text)

def cleansing(text):
    text = lowercase(text) 
    text = remove_nonaplhanumeric(text) 
    text = remove_unnecessary_char(text) 
    text = normalize_alay(text) 
    text = stemming(text) 
    text = remove_stopword(text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)

    return text

In [ ]:
df['text'] = df.text.apply(cleansing)
df.head()

In [ ]:
data_preprocessed = df.text.tolist()
type(data_preprocessed)

In [ ]:
len(data_preprocessed)

Features Extraction

In [ ]:
count_vect = TfidfVectorizer()

count_vect.fit(data_preprocessed)

X = count_vect.transform(data_preprocessed)
print("Feature Extraction Done !")

In [ ]:
import pickle

pickle.dump(count_vect, open("feature.p", "wb"))

Test Split

In [ ]:
classes = df.label
classes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size= 0.2)

In [ ]:
model_mlp = MLPClassifier()
model_mlp.fit(X_train, y_train)

print("Training Selesai !")

In [ ]:
pickle.dump(model_mlp, open("model.p", "wb"))

Evaluations

In [ ]:
test = model_mlp.predict(X_test)

print("Testing Selesai !")
print(classification_report(y_test, test))

In [ ]:
df.label.unique().tolist()

Predict

In [ ]:
original_text = '''
Amin, smg selalu di lancarkan n di mudahkan rejeki nya..
'''

text = count_vect.transform([cleansing(original_text)])

result = model_mlp.predict(text)[0]
print("Sentiment: ", result)